# Streaming

Streaming is a capability that allows AI providers to deliver responses as tokens are generated in real-time,
rather than waiting for the complete response to be finished.
You can visualize it as filling a bucket with water — streaming lets you watch the water level rising in real-time,
instead of placing the bucket somewhere, walking away, and only returning once it's completely full.

Streaming significantly improves the user experience,
as people can immediately see the response forming rather than staring at a loading indicator.
This is especially valuable since synchronous requests can take quite a long time to generate.

Let's implement streaming responses with Kotlin and Spring AI.
For this, we'll need to add dependencies.
In addition to `spring-ai-openai`, we'll need the Kotlin coroutines library to work with `Flow`.

In [1]:
%useLatestDescriptors
%use coroutines
%use spring-ai-openai
USE { dependencies { implementation("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.10.2") } }

As in the previous notebooks, we'll need an API key.

To use the model, we need to provide an API key.

You can obtain this API key from
[console.anthropic.com](https://console.anthropic.com/settings/keys)
for Anthropic models or from
[platform.openai.com](https://platform.openai.com/api-keys)
for OpenAI models.

Then add the generated API key to your environment variables:

[MacOS/Linux]
```bash
export ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
export OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI

```

[Windows]
```shell
set ANTHROPIC_API_KEY=<INSERT KEY HERE> # for Anthropic
set OPENAI_API_KEY=<INSERT KEY HERE> # for OpenAI
```

Let's retrieve the API key from environment variables:

In [2]:
val apiKey = System.getenv("OPENAI_API_KEY") ?: "YOUR_OPENAI_API_KEY"

Now let's create a `ChatModel` and pass in the `ChatOptions`:

In [3]:
val openAiApi = OpenAiApi.builder().apiKey(apiKey).build()
val openAiOptions = OpenAiChatOptions.builder()
    .model(OpenAiApi.ChatModel.GPT_5_CHAT_LATEST)
    .temperature(0.7)
    .build()

val chatCompletion = OpenAiChatModel.builder()
    .openAiApi(openAiApi)
    .defaultOptions(openAiOptions)
    .build()

Now let's ask the model to generate a short fairy tale about rabbits:

In [4]:
import kotlinx.coroutines.reactive.asFlow

val response: Flow<String> = chatCompletion.stream("Generate me 10 sentences of a fairy tale about rabbits").asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time, in a sun-dappled meadow, there lived a family of clever rabbits.  
Every morning, the youngest rabbit, Pippin, raced the wind to see who was faster.  
One day, he discovered a golden acorn hidden beneath a patch of clover.  
The acorn shimmered with magic, whispering promises of adventure and courage.  
Pippin carried it to the wise old rabbit, Willow, who lived beneath the tallest oak.  
Willow told him the acorn could grant one wish—but only if his heart was pure.  
That night, as stars danced above, Pippin wished for the meadow to stay safe forever.  
The next morning, a gentle rain fell, and the grass glowed greener than ever before.  
The foxes who once prowled nearby vanished into distant woods, never to return.  
And from that day on, the rabbits lived in peace, telling Pippin’s story under the moonlight.

As you can see, the response starts printing immediately.
If this weren't a streaming response,
we would be waiting approximately 8 seconds (judging by the cell execution time) with nothing happening.

Now let's do the same thing using the `ChatClient`:

In [5]:
val chatClient = ChatClient.create(chatCompletion)

val response: Flow<String> = chatClient
    .prompt("Generate me 10 sentences of a fairy tale about rabbits")
    .stream()
    .content()
    .asFlow()

runBlocking {
    response.collect { print(it) }
}

Once upon a time, in a sunlit meadow bordered by whispering pines, lived a family of clever rabbits.  
The youngest rabbit, named Lila, had fur as white as clouds and eyes full of curiosity.  
Every morning, she hopped beyond the burrow, eager to discover something new.  
One day, she found a mysterious golden leaf that shimmered even in the shade.  
When she touched it, a gentle voice whispered, “Follow the light, and your courage will be rewarded.”  
Lila followed the golden glow through twisting roots and mossy stones until she reached a hidden glade.  
There, a wise old owl told her that the forest’s heart had grown dim because the moonstone had been stolen.  
Determined to help, Lila gathered her bravest friends and set off on a moonlit quest.  
Together, they outwitted a sly fox, crossed a rushing brook, and found the moonstone in a hollow log.  
When they returned it to the forest’s heart, the woods sparkled once more, and the rabbits danced beneath the silver light.